<a href="https://colab.research.google.com/github/ArtDz12/Paydot/blob/main/Coding_Looping_20x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
!pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
from sklearn.metrics import r2_score
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter, YearLocator
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
# Mengimpor data (Anda harus menyesuaikan ini dengan lokasi file data Anda)
datastock = pd.read_csv('jkse20.csv', usecols=['Date', 'Close'])
datastock['Date'] = pd.to_datetime(datastock['Date'])
datastock.set_index('Date', inplace=True)

datastock.reset_index(drop=True, inplace=True)
datastock['Close'] = pd.to_numeric(datastock['Close'], errors='coerce')
datastock.fillna(datastock['Close'].mean(), inplace=True)

# Normalisasi data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(datastock)

# Membuat dataset untuk pelatihan dan pengujian
def create_dataset(datastock, look_back=1):
    X, Y = [], []
    for i in range(len(datastock)-look_back-1):
        a = datastock[i:(i+look_back), 0]
        X.append(a)
        Y.append(datastock[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 15
X, Y = create_dataset(data_scaled, look_back)

# Membagi data latih dan data uji
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(Y)]

# Mereshape input menjadi [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Inisialisasi daftar untuk menyimpan RMSE dan R²
train_scores_rmse = []
test_scores_rmse = []
train_scores_r2 = []
test_scores_r2 = []

# Loop untuk pelatihan model sebanyak 20 kali
for i in range(20):
    # Membangun model LSTM
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    # Menggunakan optimizer Adagrad dengan learning rate 0.1
    optimizer = SGD(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Melatih model
    model.fit(X_train, Y_train, epochs=40, batch_size=20, verbose=0)

    # Membuat prediksi
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # Mengembalikan skala data
    train_predict = scaler.inverse_transform(train_predict)
    Y_train_inv = scaler.inverse_transform([Y_train])
    test_predict = scaler.inverse_transform(test_predict)
    Y_test_inv = scaler.inverse_transform([Y_test])

    # Menghitung RMSE
    train_score_rmse = np.sqrt(mean_squared_error(Y_train_inv[0], train_predict[:,0]))
    test_score_rmse = np.sqrt(mean_squared_error(Y_test_inv[0], test_predict[:,0]))

    # Menghitung R²
    train_score_r2 = r2_score(Y_train_inv[0], train_predict[:,0])
    test_score_r2 = r2_score(Y_test_inv[0], test_predict[:,0])

    # Menyimpan RMSE dan R² ke dalam daftar
    train_scores_rmse.append(train_score_rmse)
    test_scores_rmse.append(test_score_rmse)
    train_scores_r2.append(train_score_r2)
    test_scores_r2.append(test_score_r2)

# Menghitung rata-rata dan standar deviasi dari RMSE dan R²
train_mean_rmse = np.mean(train_scores_rmse)
train_std_rmse = np.std(train_scores_rmse)
test_mean_rmse = np.mean(test_scores_rmse)
test_std_rmse = np.std(test_scores_rmse)

train_mean_r2 = np.mean(train_scores_r2)
train_std_r2 = np.std(train_scores_r2)
test_mean_r2 = np.mean(test_scores_r2)
test_std_r2 = np.std(test_scores_r2)

print(f'Average Train RMSE: {train_mean_rmse:.2f}')
print(f'Standard Deviation Train RMSE: {train_std_rmse:.2f}')
print(f'Average Test RMSE: {test_mean_rmse:.2f}')
print(f'Standard Deviation Test RMSE: {test_std_rmse:.2f}')

print(f'Average Train R²: {train_mean_r2:.2f}')
print(f'Standard Deviation Train R²: {train_std_r2:.2f}')
print(f'Average Test R²: {test_mean_r2:.2f}')
print(f'Standard Deviation Test R²: {test_std_r2:.2f}')

6/6 [==============================] - 0s 11ms/step
Average Train RMSE: 605.50
Standard Deviation Train RMSE: 45.50
Average Test RMSE: 524.68
Standard Deviation Test RMSE: 42.68
Average Train R²: 0.41
Standard Deviation Train R²: 0.09
Average Test R²: -12.24
Standard Deviation Test R²: 2.13
